In [1]:
import numpy as np
import pandas as pd
import ampligraph

ampligraph.__version__

'1.3.2'

In [2]:
from ampligraph.datasets import load_from_csv

# X = load_from_csv('.', '../Homework07/GoT.csv', sep=',')

In [46]:
X = load_from_csv('.', 'data/graph_relations.csv', sep=',')

In [47]:
pd.DataFrame(X, columns=['s', 'p', 'o']).head()

,s,p,o
0,Pretenders,band,is_band_true
1,Pretenders,rank-177,billboard-week-2008
2,Pretenders,rank-150,billboard-week-2016
3,Pretenders,rank-32,billboard-week-2008
4,Pretenders,rank-74,billboard-week-2008


In [48]:
hehe = [type(x) for x in X[:, 2]]

In [49]:
for i, x in enumerate(hehe):
    if "str" not in str(x):
        print(X[i])

In [50]:
from ampligraph.evaluation import train_test_split_no_unseen 

# we create a 10% test set split
X_train, X_test = train_test_split_no_unseen(X, test_size=int(X.shape[0]/10))

In [51]:
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (183406, 3)
Test set size:  (20378, 3)


In [52]:
from ampligraph.latent_features import TransE, DistMult, ComplEx

In [53]:
# Experiments: DistMult vs ComplEx

In [54]:
EmbeddingMethod = DistMult

In [75]:
model = EmbeddingMethod(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=500, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [76]:
positives_filter = X

In [77]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(X_train, early_stopping = False)

Average Loss:   0.025233: 100%|██████████| 200/200 [49:21<00:00, 14.81s/epoch]


In [78]:
from ampligraph.evaluation import evaluate_performance

In [79]:
ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=positives_filter,
                             use_default_protocol=True,
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 20378/20378 [19:07<00:00, 17.76it/s]


In [80]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.26
Hits@10: 0.46
Hits@3: 0.29
Hits@1: 0.16


In [81]:
len(model.get_embeddings(["Angie Stone"], embedding_type="entity")[0])

500

In [82]:
import pickle
with open('data/artist_names.p', 'rb') as f:
    artist_name = pickle.load(f)

In [83]:
model = EmbeddingMethod(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=500, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [84]:
model.fit(X, early_stopping = False)

Average Loss:   0.028267: 100%|██████████| 200/200 [17:48<00:00,  5.34s/epoch]


In [85]:
embeddings = model.get_embeddings(artist_name, embedding_type="entity")

In [86]:
embeddings.shape

(5370, 500)

In [87]:
with open("data/embeddings.p", "wb") as f:
    pickle.dump(embeddings, f)

In [88]:
artist_dict = {}
for i, name in enumerate(artist_name):
    artist_dict[name] = i

In [89]:
with open('data/artist_dict.p', 'wb') as f:
    pickle.dump(artist_dict, f)